세팅

In [13]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
import shutil

# 원본 데이터셋 경로
source_dataset_path  = '/content/drive/MyDrive/Python_project/Data/current_dataset'
resized_dataset_path = '/content/drive/MyDrive/Python_project/Data/resized_dataset'
model_path = '/content/drive/MyDrive/딥러닝프로젝트_쓰레기분류모델_CNN/data/model'

# 쓰레기 카테고리
categories = ['glass','metal','paper','plastic','vinyl']
sizes = []

# 테스트를 위해 갯수 제한
# images_per_category = 300

# 이미지 리사이즈 크기 결정
resize_width, resize_height = 384, 512

리사이즈 이미지 설정

In [14]:
for category in categories:
    category_path = os.path.join(source_dataset_path, category)
    # 카테고리 폴더 내의 이미지 중 png,jpg,jpeg 필터링
    # 앞의 f 필터링된 파일이름
    # 뒤의 f 카테고리 폴더의 모든 파일
    # f.lower().endswith(('.png', '.jpg', '.jpeg') -> 파일이름을 소문자로 변경하고
    # 해당 이름이 '.png', '.jpg', 또는 '.jpeg'로 끝나는지를 검사
    image_files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f)) and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # 각 카테고리별로 지정된 수의 이미지만 처리
    for filename in image_files[:]:# images_per_category
        file_path = os.path.join(category_path, filename)
        # OpenCV를 사용하여 이미지를 로드하고, 이미지의 크기를 sizes 리스트에 추가합니다.
        image = cv2.imread(file_path)
        if image is not None:
            sizes.append(image.shape[:2])  # 이미지의 높이와 너비만 추출

In [15]:
# 이미지 크기 통계 계산
if sizes:
    heights, widths = zip(*sizes)
    avg_height = sum(heights) / len(heights)
    avg_width = sum(widths) / len(widths)
    min_height = min(heights)
    min_width = min(widths)
    median_height = sorted(heights)[len(heights) // 2]
    median_width = sorted(widths)[len(widths) // 2]
else:
    avg_height = avg_width = min_height = min_width = median_height = median_width = 0

(avg_height, avg_width), (min_height, min_width), (median_height, median_width)

((366.48687448728464, 472.9179655455291), (159, 177), (384, 512))

이미지 리사이즈

In [16]:
# 카테고리별로 폴더를 순회하며 이미지 처리
for category in categories:
    source_category_path = os.path.join(source_dataset_path, category)
    resized_category_path = os.path.join(resized_dataset_path, category)
    os.makedirs(resized_category_path, exist_ok=True)  # 리사이즈된 이미지 저장 폴더 생성
    # exist_ok=True 해당 디렉토리에 폴더가 존재해도 오류를 발생시키지 않고 넘어감

    # 이미지 파일 처리
    # images_per_category
    image_files = os.listdir(source_category_path)[:]  # 각 폴더별로 처음 50개의 파일만 가져옴
    for filename in image_files:
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(source_category_path, filename)
            image = cv2.imread(file_path)
            if image is not None:
                resized_image = cv2.resize(image, (resize_width, resize_height))
                cv2.imwrite(os.path.join(resized_category_path, filename), resized_image)

데이터 전처리 및 분할

In [17]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # 이미지 회전 범위
    width_shift_range=0.1,  # 수평 이동 범위
    height_shift_range=0.1,  # 수직 이동 범위
    shear_range=0.1,  # 전단 변환 범위
    zoom_range=0.1,  # 확대/축소 범위
    horizontal_flip=True,  # 수평 뒤집기 적용
    fill_mode='nearest',
    validation_split=0.2  # 데이터셋의 20%를 검증 데이터로 사용
)

# 참고 : https://tykimos.github.io/2017/06/10/CNN_Data_Augmentation/

# 학습 데이터셋 로더 설정
train_generator = train_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=32, # 20~30
    class_mode='categorical',
    subset='training'  # 학습 데이터셋
)

# 검증 데이터셋 로더 설정
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 검증 데이터셋도 정규화 필요
validation_generator = validation_datagen.flow_from_directory(
    resized_dataset_path,
    target_size=(resize_width, resize_height),
    batch_size=32, # 20~30
    class_mode='categorical',
    subset='validation'  # 검증 데이터셋
)

Found 1952 images belonging to 5 classes.
Found 486 images belonging to 5 classes.


In [18]:
len(train_generator)

61

In [19]:
len(validation_generator)

16

모델 정의 (Define Network)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 모델 구축
model = Sequential([
    # 컨볼루션 및 풀링 레이어
    Conv2D(32, (3, 3), activation='relu', input_shape=(resize_width, resize_height, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    # 드롭아웃 추가
    Dropout(0.1),
    # Flatten 및 Dense 레이어
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # 클래스가 5개인 경우
])

# 모델 요약
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 382, 510, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 191, 255, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 189, 253, 64)      18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 94, 126, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 92, 124, 128)      73856     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 46, 62, 128)      

모델 컴파일 (Compile Network)
keras코드 -> tensorflow 코드로 변환

In [21]:
from tensorflow.keras.optimizers import Adam

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

모델 학습 (Fit Network)

In [22]:
# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),  # 학습 데이터셋의 이미지 수를 배치 크기로 나눈 값
    epochs=70,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)  # 검증 데이터셋의 이미지 수를 배치 크기로 나눈 값
)
# 수업 때 배운 걸로 모델 저장하면서 학습
model_save_path = os.path.join(model_path, 'my_model.h5')
model.save(model_save_path)

Epoch 1/70
61/61 [==============================] - 87s 1s/step - loss: 1.4974 - accuracy: 0.3484 - val_loss: 1.2490 - val_accuracy: 0.4733
Epoch 2/70
61/61 [==============================] - 82s 1s/step - loss: 1.2951 - accuracy: 0.4513 - val_loss: 1.2581 - val_accuracy: 0.4568
Epoch 3/70
61/61 [==============================] - 82s 1s/step - loss: 1.1899 - accuracy: 0.5231 - val_loss: 1.0514 - val_accuracy: 0.5638
Epoch 4/70
61/61 [==============================] - 82s 1s/step - loss: 1.0485 - accuracy: 0.5876 - val_loss: 0.9945 - val_accuracy: 0.5700
Epoch 5/70
61/61 [==============================] - 83s 1s/step - loss: 0.9807 - accuracy: 0.6183 - val_loss: 1.0100 - val_accuracy: 0.5885
Epoch 6/70
61/61 [==============================] - 81s 1s/step - loss: 0.9552 - accuracy: 0.6301 - val_loss: 1.0727 - val_accuracy: 0.5679
Epoch 7/70
61/61 [==============================] - 82s 1s/step - loss: 1.0330 - accuracy: 0.5856 - val_loss: 0.9657 - val_accuracy: 0.6420
Epoch 8/70
61/61 [==

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델 평가 (Evaluate Network)

In [23]:
# 모델 평가
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

16/16 [==============================] - 4s 212ms/step - loss: 0.9977 - accuracy: 0.7881
Validation Loss: 0.9977259635925293
Validation Accuracy: 0.7880658507347107


예측 수행 (Make Predictions): 쓰레기 이미지를 넣어 쓰레기의 종류를 분류

In [24]:
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# 이미지를 불러오고 전처리하는 함수
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(resize_width, resize_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델의 예상 입력 형태에 맞게 차원 추가
    img_array /= 255.0  # 이미지 정규화
    return img_array

# 테스트 이미지 폴더 경로
test_images_path = '/content/drive/MyDrive/Python_project/Data/img'

# 테스트 이미지 파일 목록 가져오기
test_image_files = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# 각 이미지에 대해 예측 수행
for image_file in test_image_files:
    img_array = load_and_preprocess_image(image_file)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = categories[predicted_class_index]
    print(f"Image: {image_file}, Predicted class: {predicted_class_name}")


1/1 [==============================] - 0s 496ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/metal28.jpg, Predicted class: metal
1/1 [==============================] - 0s 18ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/metal19.jpg, Predicted class: metal
1/1 [==============================] - 0s 19ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/glass26.jpg, Predicted class: glass
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/metal1.jpg, Predicted class: glass
1/1 [==============================] - 0s 21ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/metal10.jpg, Predicted class: glass
1/1 [==============================] - 0s 20ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/glass18.jpg, Predicted class: glass
1/1 [==============================] - 0s 18ms/step
Image: /content/drive/MyDrive/Python_project/Data/img/glass10.jpg, Predicted class: vinyl
1/1 [=

In [25]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# 이미지를 불러오고 전처리하는 함수
def load_and_preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(resize_width, resize_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # 모델의 예상 입력 형태에 맞게 차원 추가
    img_array /= 255.0  # 이미지 정규화
    return img_array

# 디렉토리 이름에서 레이블 추출
def extract_label_from_directory(file_path):
    return os.path.basename(os.path.dirname(file_path))

# 테스트 이미지 폴더 경로
test_images_path = '/content/drive/MyDrive/Python_project/Data/img'

# 테스트 이미지 파일 목록 가져오기
test_image_files = [os.path.join(test_images_path, f) for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# 각 이미지에 대해 예측 수행 및 시각화
for image_file in test_image_files:
    true_label = extract_label_from_directory(image_file)
    img_array = load_and_preprocess_image(image_file)
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class_name = categories[predicted_class_index]

    # 시각화
    img = plt.imread(image_file)
    plt.imshow(img)
    plt.title(f"Predicted: {predicted_class_name}")
    plt.axis('off')
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [26]:
import os
import re
from tensorflow.keras.preprocessing import image
import numpy as np

# 이미지를 불러오고 전처리하는 함수
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(resize_width, resize_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    return img_array

# 파일 이름에서 레이블 추출
def get_label_from_filename(file_name):
    if re.match(r'glass\d+', file_name):
        return 'glass'
    elif re.match(r'metal\d+', file_name):
        return 'metal'
    elif re.match(r'paper\d+', file_name):
        return 'paper'
    elif re.match(r'plastic\d+', file_name):
        return 'plastic'
    elif re.match(r'\d+', file_name):  # 숫자로만 이루어진 파일 이름 처리
        return 'vinyl'
    elif re.match(r'trach\d+', file_name):  # 'trach'로 시작하는 파일 이름 처리
        return 'vinyl'
    else:
        return 'unknown'  # 알려지지 않은 패턴 처리

# 모델 예측 및 정확도 계산
def evaluate_model_accuracy(model, base_path, categories):
    correct_predictions = 0
    total_images = 0

    # base_path 폴더 내의 모든 파일을 검색
    for file_name in os.listdir(base_path):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            # 레이블 추출 및 이미지 전처리
            true_label = get_label_from_filename(file_name)
            if true_label != 'unknown':  # 알 수 없는 레이블을 제외
                img_path = os.path.join(base_path, file_name)
                img_array = load_and_preprocess_image(img_path)

                # 예측 실행
                prediction = model.predict(img_array)
                predicted_label_index = np.argmax(prediction[0])
                predicted_label = categories[predicted_label_index]

                # 예측 및 실제 레이블 출력
                print(f"Actual: {true_label}, Predicted: {predicted_label}")

                # 예측 정확도 업데이트
                if predicted_label == true_label:
                    correct_predictions += 1
                total_images += 1

    # 정확도 계산 및 출력
    accuracy = correct_predictions / total_images if total_images > 0 else 0
    return accuracy

# 모델 정확도 평가
base_path = '/content/drive/MyDrive/Python_project/Data/img'
categories = ['glass', 'metal', 'paper', 'plastic', 'vinyl']  # 모델이 예측하는 클래스 목록
accuracy = evaluate_model_accuracy(model, base_path, categories)
print("\n")
print(f"Model Accuracy: {accuracy * 100:.2f}%")


1/1 [==============================] - 0s 19ms/step
Actual: metal, Predicted: metal
1/1 [==============================] - 0s 18ms/step
Actual: metal, Predicted: metal
1/1 [==============================] - 0s 18ms/step
Actual: glass, Predicted: glass
1/1 [==============================] - 0s 18ms/step
Actual: metal, Predicted: glass
1/1 [==============================] - 0s 18ms/step
Actual: metal, Predicted: glass
1/1 [==============================] - 0s 18ms/step
Actual: glass, Predicted: glass
1/1 [==============================] - 0s 18ms/step
Actual: glass, Predicted: vinyl
1/1 [==============================] - 0s 18ms/step
Actual: glass, Predicted: glass
1/1 [==============================] - 0s 17ms/step
Actual: glass, Predicted: glass
1/1 [==============================] - 0s 17ms/step
Actual: paper, Predicted: paper
1/1 [==============================] - 0s 18ms/step
Actual: paper, Predicted: paper
1/1 [==============================] - 0s 18ms/step
Actual: paper, Predicted